In [2]:
from os.path import expanduser, join
root = join(expanduser("~"), 't.ex-Graph')

In [3]:
import sys
sys.path.insert(1, join(root, 'lib'))

import config
import functions
import data
import model

In [4]:
datasets = [
  data.read(join(root, 'data', 'graph-data-sld.csv')),
  data.read(join(root, 'data', 'graph-data-fqdn.csv')),
]

In [5]:
features = [col for col in list(datasets[0].columns) if col.lower() not in ['id', 'weight', 'tracker']]

In [6]:
from sklearn.preprocessing import LabelEncoder

for dataset in datasets:
  dataset['tracker'] = LabelEncoder().fit_transform(dataset['tracker'])

In [7]:
extension = []
for dataset in datasets:
  extension.append(data.sample_equal_distribution(dataset, 'tracker'))

datasets.extend(extension)

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier

models = {
  'continuous': [
    LinearRegression(),
    RandomForestRegressor(n_estimators=200, random_state=0, n_jobs=-1)
  ],
  'category':[
    DecisionTreeClassifier(),
    LogisticRegression(solver='lbfgs', max_iter=1000)
  ]
}

In [ ]:
for target in ['weight', 'tracker']:
  for dataset in datasets:
    x = model.run_tests(models, dataset, features, target)
    print (x)